First, we start of by importing the necessary python libraries.

In [258]:
import pandas as pd
import numpy as np

Next step is importing our data files and running transformation operations, we will start by removing the unnecessary columns

In [259]:
movies_revenues=pd.read_csv("movies_metadata.csv")
engagement=pd.read_excel("netflix_engagement_report.xlsx")
netflixlib=pd.read_csv("netflix_library.csv")
ratings= pd.read_table("imdb_ratings.tsv")
netflixlibcredits=pd.read_csv("netflix_library_credits.csv")

movies_revenues.drop(['adult','runtime','original_language','production_companies','belongs_to_collection','genres','homepage','id', 'runtime',
              'original_title','overview','popularity',
              'poster_path','production_countries',
              'spoken_languages','status','tagline','video','title','vote_average','vote_count','release_date'],
              axis=1, inplace=True)
netflixlib.drop(['description','imdb_score','imdb_votes','tmdb_score','seasons','age_certification','tmdb_popularity'], axis=1, inplace=True )
netflixlibcredits.drop(['character'], axis=1, inplace=True )
engagement.drop('release_date', axis=1, inplace=True )

C:\Users\aziba\AppData\Local\Temp\ipykernel_11476\3671836542.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_revenues=pd.read_csv("movies_metadata.csv")


1. netflix_engagement_report:
Transforming the netflix engagement report to match the title names in our dataset and calculate the total sum of hours viewed for each show/movie, this will be our fact table later

In [260]:
transformed_titles = []
for title in engagement['title']:
    # Remove ": Season" and everything after it
    if ': Season' in title:
        title = title.split(': Season')[0].strip()
    # Remove "//" and everything after it
    if '//' in title:
        title = title.split('//')[0].strip() 
    # Remove year in parentheses
    title = title.split('(')[0].strip() 
    # Add the transformed title to the list
    transformed_titles.append(title)
# replace the titles with their new values
engagement['title'] = transformed_titles
# calculate the sum of hours_viewed for each show across their different seasons
engagement_grouped = engagement.groupby('title').agg({'hours_viewed':'sum','global_availability':'first'}).reset_index().sort_values(by='hours_viewed',ascending=False)
engagement_grouped.insert(0, 'engagement_id', range(1, len(engagement_grouped) + 1))
display(engagement_grouped)
engagement_grouped.info()

,engagement_id,title,hours_viewed,global_availability
4565,1,Ginny & Georgia,967200000,Yes
12911,2,The Night Agent,812100000,Yes
15029,3,You,766300000,Yes
9057,4,Outer Banks,740400000,Yes
13536,5,The Walking Dead,738600000,No
...,...,...,...,...
6689,15231,Lagenda Budak Hostel,100000,No
6677,15232,Ladies in Black,100000,No
6676,15233,Ladies Up,100000,Yes
12348,15234,The Grown-Ups,100000,No


<class 'pandas.core.frame.DataFrame'>
Index: 15235 entries, 4565 to 12903
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   engagement_id        15235 non-null  int64 
 1   title                15235 non-null  object
 2   hours_viewed         15235 non-null  int64 
 3   global_availability  15235 non-null  object
dtypes: int64(2), object(2)
memory usage: 595.1+ KB


2. netflix_library_credits: we are going to make each row unique, as well as making dictionary type columns for actors and directors separeatly

In [261]:
netflixlibcredits['casting_id'] = netflixlibcredits.reset_index().index + 1 
#renaming the dataframe temporarily for more simplicity
df=netflixlibcredits
# Group by tmdb_id, correctly filtering roles for actors and directors
grouped_df = df.groupby('tmdb_id').agg(
    actors=('person_id', lambda x: dict(zip(x[df.loc[x.index, 'role'] == 'ACTOR'], df.loc[x.index[df.loc[x.index, 'role'] == 'ACTOR'], 'name']))),
    directors=('person_id', lambda x: dict(zip(x[df.loc[x.index, 'role'] == 'DIRECTOR'], df.loc[x.index[df.loc[x.index, 'role'] == 'DIRECTOR'], 'name'])))
)
# Reset the index and handle cases without directors
result_df = grouped_df.reset_index().fillna({'directors': {}})  # Fill missing directors with empty dict
result_df = result_df[result_df['directors'].astype(bool)]
result_df = result_df.reset_index(drop=True)
netflixlibcredits=result_df
netflixlibcredits.insert(0, 'cast_id', range(1, len(netflixlibcredits) + 1))
display(netflixlibcredits)
netflixlibcredits.info()


,cast_id,tmdb_id,actors,directors
0,1,tm1000037,"{251702: 'Luna Wedler', 41192: 'Jannis Niewöhn...",{142171: 'Christian Schwochow'}
1,2,tm1000147,"{5080: 'Guy Pearce', 159223: 'Matilda Anna Ing...",{954232: 'Andrew Baird'}
2,3,tm100015,"{1675: 'Idris Elba', 2673: 'Paul Walker', 1313...",{13150: 'John Luessenhop'}
3,4,tm1000166,"{248002: 'Glenn Fredly', 1793972: 'Marcello Ta...",{1793832: 'Saron Sakina'}
4,5,tm1000185,"{1562688: 'Adrianna Chlebicka', 299215: 'Mateu...",{446979: 'Filip Zylber'}
...,...,...,...,...
4036,4037,ts91039,"{1681: 'Jon Favreau', 16294: 'Roy Choi'}",{1681: 'Jon Favreau'}
4037,4038,ts91124,"{98235: 'Kim So-hyun', 736806: 'Jung Ga-ram', ...",{671210: 'Kim Jin-woo'}
4038,4039,ts92547,"{2216637: 'Ahmet Kürşat Öçalan', 200830: 'Daml...",{1322983: 'Emre Sahin'}
4039,4040,ts94677,"{2285125: 'John Cabrera', 2285176: 'Judy', 228...",{242191: 'Domini Hofmann'}


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4041 entries, 0 to 4040
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cast_id    4041 non-null   int64 
 1   tmdb_id    4041 non-null   object
 2   actors     4041 non-null   object
 3   directors  4041 non-null   object
dtypes: int64(1), object(3)
memory usage: 126.4+ KB


3. movies_revenues: We're going to clean the data as it has several missing values that we are not going to be interested in and that will skew our data, we can also add a column that calculates the profits

In [262]:
#remove rows with values of 0 in budget, revenue, and runtime.
movies_revenues = movies_revenues[(movies_revenues['budget'] != '0') & (movies_revenues['revenue'] != 0) ]
#remove rows with empty values
movies_revenues = movies_revenues.dropna(subset=['budget', 'revenue'], how='any')
#changing the datatype of the budget column
movies_revenues['budget'] = movies_revenues['budget'].astype(float)
#adding the profit column
movies_revenues['profit'] = movies_revenues['revenue'] - movies_revenues['budget']
movies_revenues.insert(0, 'revenue_id', range(1, len(movies_revenues) + 1))
display(movies_revenues)
movies_revenues.info() 

,revenue_id,budget,imdb_id,revenue,profit
0,1,30000000.0,tt0114709,373554033.0,343554033.0
1,2,65000000.0,tt0113497,262797249.0,197797249.0
3,3,16000000.0,tt0114885,81452156.0,65452156.0
5,4,60000000.0,tt0113277,187436818.0,127436818.0
8,5,35000000.0,tt0114576,64350171.0,29350171.0
...,...,...,...,...,...
45250,5377,12000000.0,tt0479751,19000000.0,7000000.0
45399,5378,750000.0,tt3805180,3.0,-749997.0
45409,5379,800000.0,tt0933361,1328612.0,528612.0
45412,5380,2000000.0,tt1718881,1268793.0,-731207.0


<class 'pandas.core.frame.DataFrame'>
Index: 5381 entries, 0 to 45422
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   revenue_id  5381 non-null   int64  
 1   budget      5381 non-null   float64
 2   imdb_id     5381 non-null   object 
 3   revenue     5381 non-null   float64
 4   profit      5381 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 252.2+ KB


4. netflix_library: We are going to simply remove some rows with missing values that might cause problem in data analysis and modelling

In [263]:
netflixlib = netflixlib.dropna(subset=['imdb_id'], how='any')
netflixlib.insert(0, 'movie_show_id', range(1, len(netflixlib) + 1))
display(netflixlib)
netflixlib.info()

,movie_show_id,tmdb_id,title,type,release_year,runtime,genres,production_countries,imdb_id
1,1,tm84618,Taxi Driver,MOVIE,1976,114,"['drama', 'crime']",['US'],tt0075314
2,2,tm154986,Deliverance,MOVIE,1972,109,"['drama', 'action', 'thriller', 'european']",['US'],tt0068473
3,3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,91,"['fantasy', 'action', 'comedy']",['GB'],tt0071853
4,4,tm120801,The Dirty Dozen,MOVIE,1967,150,"['war', 'action']","['GB', 'US']",tt0061578
5,5,ts22164,Monty Python's Flying Circus,SHOW,1969,30,"['comedy', 'european']",['GB'],tt0063929
...,...,...,...,...,...,...,...,...,...
5843,5443,tm1097142,My Bride,MOVIE,2021,93,"['romance', 'comedy', 'drama']",['EG'],tt14216488
5845,5444,tm1014599,Fine Wine,MOVIE,2021,100,"['romance', 'drama']",['NG'],tt13857480
5846,5445,tm898842,C/O Kaadhal,MOVIE,2021,134,['drama'],[],tt11803618
5847,5446,tm1059008,Lokillo,MOVIE,2021,90,['comedy'],['CO'],tt14585902


<class 'pandas.core.frame.DataFrame'>
Index: 5447 entries, 1 to 5849
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   movie_show_id         5447 non-null   int64 
 1   tmdb_id               5447 non-null   object
 2   title                 5446 non-null   object
 3   type                  5447 non-null   object
 4   release_year          5447 non-null   int64 
 5   runtime               5447 non-null   int64 
 6   genres                5447 non-null   object
 7   production_countries  5447 non-null   object
 8   imdb_id               5447 non-null   object
dtypes: int64(3), object(6)
memory usage: 425.5+ KB


5. imdb_ratings:

In [264]:
ratings = ratings.rename(columns={"averageRating": "average_rating"})
ratings = ratings.rename(columns={"numVotes": "num_votes"})
display(ratings)
ratings.info()

,imdb_id,average_rating,num_votes
0,tt0000001,5.7,2014
1,tt0000002,5.7,270
2,tt0000003,6.5,1938
3,tt0000004,5.5,178
4,tt0000005,6.2,2716
...,...,...,...
1392734,tt9916730,7.6,11
1392735,tt9916766,7.1,23
1392736,tt9916778,7.2,36
1392737,tt9916840,8.8,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392739 entries, 0 to 1392738
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   imdb_id         1392739 non-null  object 
 1   average_rating  1392739 non-null  float64
 2   num_votes       1392739 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.9+ MB


6. Final step: We will export our data files into JSON files in a separate folder

In [265]:
import os
if not os.path.exists("new_data"):
    os.makedirs("new_data")
    
engagement_grouped.to_csv('new_data/new_netflix_engagement_report.csv',index=False)
movies_revenues.to_csv('new_data/new_movies_revenues.csv',index=False)
netflixlib.to_csv('new_data/new_netflix_library.csv',index=False)
ratings.to_csv('new_data/new_imdb_ratings.csv',index=False)
netflixlibcredits.to_csv('new_data/new_netflix_library_credits.csv',index=False)